In [9]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-upswell-vlab', override=True)
db_conf = get_db_conf(env)

In [7]:
USER = "malaria2@vlab.digital"
CAMPAIGN = "mnm-individual-effect"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [9]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [10]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [5]:
# make creatives conf
from adopt.marketing import CreativeConf, CampaignConf


In [ ]:
16000000.0 / 7300

In [ ]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 16000000.0,
     'min_budget': 10000.0,
     'opt_window': 2*24,
     'end_date': '2021-03-10',
     'page_id': '102998371752603',
     'instagram_id': None,
     'proportional': True,
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-individual-effect'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [7]:
import json
from adopt.marketing import make_audience_conf, dict_from_nested_type

audience_confs = [
  {
    "name": "vlab-mnm-kutcha",
    "shortcodes": ["extrabasehin"],
    "subtype": "LOOKALIKE",
    "lookalike": {
      "name": "vlab-mnm-kutcha-lookalike",
      "target": 2000,
      "spec": {
        "country": "IN",
        "starting_ratio": 0.0,
        "ratio": 0.2
      }
    },
    "question_targeting": {
      "op": "equal",
      "vars": [
        {"type": "response", "value": "dwelling"},
        {"type": "constant", "value": "A"}
      ]
    }
  },
  {
    "name": "vlab-mnm-respondents-round-B",
    "shortcodes": ["extrabasehin"],
    "subtype": "CUSTOM"
  }
]



confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

('b3bc209a-f0cb-4d6d-8164-f9f72dbb36c6',
 datetime.datetime(2021, 2, 25, 12, 24, 32, 386720, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'audience',
 None,
 [{'lookalike': {'name': 'vlab-mnm-kutcha-lookalike',
    'spec': {'country': 'IN', 'ratio': 0.2, 'starting_ratio': 0.0},
    'target': 2000},
   'name': 'vlab-mnm-kutcha',
   'question_targeting': {'op': 'equal',
    'vars': [{'type': 'response', 'value': 'dwelling'},
     {'type': 'constant', 'value': 'A'}]},
   'shortcodes': ['extrabasehin'],
   'subtype': 'LOOKALIKE'},
  {'name': 'vlab-mnm-respondents-round-B',
   'shortcodes': ['extrabasehin'],
   'subtype': 'CUSTOM'}])

In [ ]:
from adopt.marketing import CreativeConf

def _creative_conf(name, image, body, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "welcome_message": body,
        "link_text": "Messenger main survey shuroo karen!",
        "button_text": "Start!",
        "form": form,
    }

image_confs = [
    ('vlab-mnm-mother-daughter-voice-be-heard', '1.png', 'अपने विचार दुनिया को बताएं!'),
    ('vlab-mnm-family-flowers-keep-healthy', '2.png', 'हमें अपने स्वस्थ जीवन के राज बताएं और फ्री डाटा क्रेडिट जीतें।'),
    # ('vlab-mnm-schoolgirl-laptop-got-10-min', '3.png', 'क्या आपके पास सिर्फ १० मिनिट हैं? मोबाइल डाटा क्रेडिट में २०० रुपये पाएँ। आपकी राय हमारे लिए महत्वपूर्ण है!'),
    ('vlab-mnm-grandfather-bike-healthy', '4.png', 'अपने कम्यूनिटी को स्वस्थ रखने में हमें मदद करें- हमें बताएं आप क्या सोचते है!'),
    ('vlab-mnm-woman-sari-texting-got-time', '5.png', 'एक छोटे से सर्वे के लिए आपके पास टाइम है?'),
]

creatives = [CreativeConf(**_creative_conf(name, images[n], body, 'extrabasehin')) for name, n, body in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [16]:
from itertools import product
from functools import reduce

import pandas as pd
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_treatment_targeting(df):
    df = df[df.treatment == df.treatment.iloc[0]]
    ids = df.disthash.unique()

    vars_ = [{'op': 'equal', 
              'vars': [{'type': 'response', 'value': 'md:stratumid'}, 
                       {'type': 'constant', 'value': i}]}
             for i in ids]

    return {'op': 'or', 'vars': vars_}


def format_group(g):
    kw, fb, qu = zip(*g)

    combine = lambda a, b: {**a, **b}
    kw = reduce(combine, kw)
    fb = reduce(combine, fb)
    return {**kw,
            'id_': f'kutcha-{int(kw["kutcha"])}-treatment-{kw["treatment"]}-gender-{fb["genders"][0]}-age-{fb["age_min"]}',
            'quota': 100,
            'facebook_targeting': fb,
            'question_targeting': [q for q in qu if q]}



def make_stratum(df, id_, kutcha, treatment, quota, facebook_targeting, question_targeting):
    df = df[df.treatment == treatment].reset_index(drop=True)

    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    # Deal with audiences for kutcha...    
    excluded_audiences = ["vlab-mnm-respondents-panel", 
                          "vlab-mnm-respondents-round-B", 
                          "vlab-mnm-respondents-1shot", 
                          "vlab-mnm-respondents-1shot-global-insights"]
    audiences = []
    if kutcha:
        audiences = ["vlab-mnm-kutcha-lookalike"]
    else:
        excluded_audiences += ["vlab-mnm-kutcha-lookalike"]

    question_targeting = {'op': 'or',
                          'vars': [
                              {'op': 'equal', 
                               'vars': [{'type': 'response', 'value': 'md:stratumid'}, 
                                        {'type': 'constant', 'value': id_}]},
                              {'op': 'and', 
                               'vars': [make_treatment_targeting(df), 
                                        *question_targeting]}
                          ]}

    
    conf = { 
        'id': id_,
        'quota': quota,
        'audiences': audiences,
        'excluded_audiences': excluded_audiences,
        'metadata': {'stratumid': id_},
        'facebook_targeting': targeting, 
        'question_targeting': question_targeting,
        'creatives': ['vlab-mnm-mother-daughter-voice-be-heard',
                      'vlab-mnm-family-flowers-keep-healthy',
                      # 'vlab-mnm-schoolgirl-laptop-got-10-min',
                      'vlab-mnm-grandfather-bike-healthy',
                      'vlab-mnm-woman-sari-texting-got-time'],
        'shortcodes': ['extrabasehin'],}

    return conf



In [17]:
groups = [
    [
        ({}, {'genders': [1]}, {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'gender'}, 
                                                    {'type': 'constant', 'value': 'Male'}]}), 
        ({}, {'genders': [2]}, {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'gender'}, 
                                                    {'type': 'constant', 'value': 'Female'}]}),
    ],
    [
        ({}, {'age_min': 20, 'age_max': 25}, 
         {'op': 'less_than', 'vars': [{'type': 'response', 'value': 'age'},
                                      {'type': 'constant', 'value': 26}]}), 

        ({}, {'age_min': 26, 'age_max': 39}, 
         {'op': 'and', 'vars': [
             {'op': 'greater_than', 'vars': [{'type': 'response', 'value': 'age'},
                                            {'type': 'constant', 'value': 25}]}, 
             {'op': 'less_than', 'vars': [{'type': 'response', 'value': 'age'},
                                         {'type': 'constant', 'value': 40}]}]}),

        ({}, {'age_min': 40}, 
         {'op': 'greater_than', 'vars': [{'type': 'response', 'value': 'age'},
                                         {'type': 'constant', 'value': 39}]}), 
    ],
    [
        ({'treatment': 1}, {}, {}),
        ({'treatment': 0}, {}, {}),
    ],
    [
        ({'kutcha': True}, {}, { 'op': 'equal',
                                 'vars': [{'type': 'response', 'value': 'dwelling'},
                                          {'type': 'constant', 'value': 'A'}]}),
        ({'kutcha': False}, {},  { 'op': 'not_equal',
                                 'vars': [{'type': 'response', 'value': 'dwelling'},
                                          {'type': 'constant', 'value': 'A'}]}),
    ]    
]

treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
df = load_cities('output/cities.csv')
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]
df = df.merge(treatment_assignment)

confs = [format_group(g) for g in product(*groups)]
stratum_confs = [make_stratum(df, **c) for c in confs]
stratum_confs = [dict_from_nested_type(make_stratum_conf(c)) for c in stratum_confs]

In [ ]:
create_campaign_confs(CAMPAIGNID, "stratum", stratum_confs, db_conf)

# TESTING

In [224]:
mal = load_basics(CAMPAIGNID, env)

In [45]:
from adopt.malaria import hydrate_audiences, get_df

userinfo, config, db_conf, state, m, confs = mal
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
from adopt.malaria import update_audience_for_campaign, update_ads_for_campaign

# instructions = update_audience_for_campaign(mal)
instructions, report = update_ads_for_campaign(mal)

In [19]:
from adopt.malaria import hydrate_strata, hydrate_audiences
from adopt.malaria import run_instructions

# run_instructions(instructions, mal.state)

In [11]:
import pandas as pd
from adopt.campaign_queries import get_last_adopt_report

rdf = pd.DataFrame(get_last_adopt_report(CAMPAIGNID, "FACEBOOK_ADOPT", mal.db_conf)).T

In [ ]:
# select current_state, fb_error_code, state_json->'previousOutput' from states where current_form = 'extrabasehin' and timeout_date is not null;

# select timeout_date, current_state, fb_error_code, state_json->'tokens'->>0, state_json->'previousOutput'->>'token', state_json->'error', state_json->'previousOutput' from states where current_form = 'extrabasehin' and fb_error_code = '2022' and state_json->'previousOutput'->>'token' is not null;

# select fb_error_code, count(*) from states where current_form = 'extrabasehin' and (state_json->'previousOutput'->>'token' is not null or state_json->'tokens'->>0 is not null) group by fb_error_code;

# 

In [225]:
from adopt.marketing import create_custom_audience, add_users_to_audience


name = 'vlab-mnm-individual-effect-retargeting'
# i = create_custom_audience(name, 'Audience for retargeting malaria ad campaign to test individual effects')

audid = next(aud['id'] for aud in mal.state.custom_audiences if aud['name'] == name)

In [4]:
from adopt.responses import query

q = """
    WITH t AS (
      SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY question_ref, userid, surveyid
                           ORDER BY timestamp DESC) as n
      FROM (select * from states join responses using(userid) where current_form = 'extrabasehin' and (state_json->'previousOutput'->>'token' is not null or state_json->'tokens'->>0 is not null) and (fb_error_code != '551' or fb_error_code is null))
      WHERE shortcode = 'extrabasehin'
    )
    SELECT metadata, userid, surveyid, shortcode, question_ref,
           response, timestamp, translated_response

    FROM t WHERE n = 1
"""


q = """
    WITH t AS (
      SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY question_ref, userid, surveyid
                           ORDER BY timestamp DESC) as n
      FROM (select * from states join responses using(userid) where current_form = 'extrabasehin' and (state_json->'previousOutput'->>'token' is not null or state_json->'tokens'->>0 is not null) and (fb_error_code != '551' or fb_error_code is null))
      WHERE shortcode = 'extrabasehin'
    )
    SELECT metadata, userid, surveyid, shortcode, question_ref,
           response, timestamp, translated_response

    FROM t WHERE n = 1
"""

user_data = [d for d in query(db_conf, q, (), as_dict=True)]

In [ ]:
# TODO: Bail out everyone who doesn't get timeout on April 10th!

In [15]:
df = pd.DataFrame(user_data)

In [18]:
df.userid.unique().shape

(4345,)

In [206]:
from mnm.prep import get_form_data, format_response_df, pivot_response_df

metadata_keys = ['stratumid']
tester_key = 'stratumid'
forms, form_keys = get_form_data(['MNM', 'malaria no more round b 1-shot'], db_conf)
rdf = format_response_df(df, forms, form_keys, metadata_keys, tester_key)
rdf = pivot_response_df(
    rdf, metadata_keys, form_keys, ["shortcode"]
)

rdf = rdf[rdf.dwelling.notna()]

In [207]:
import re

def get_treatment(lookup, stratumid):
    match = re.search('treatment-(\d)-', stratumid)
    if match:
        return int(match[1])
    return lookup[stratumid]

treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
treatment_assignment = treatment_assignment.set_index('disthash').treatment.to_dict()
    
rdf['treatment'] = rdf.stratumid.map(lambda s: get_treatment(treatment_assignment, s))

In [ ]:
# age, gender, treatment, dwelling = kutcha

rdf[['age', 'gender', 'dwelling', 'treatment', 'malaria4months']]

In [ ]:
def binarize(rdf, conf):
    pass

conf = [
    ('female', [('gender', 'Female')])
]

In [188]:
rdf['female'] = rdf.gender == 'Female'
rdf['kutcha'] = rdf.dwelling == 'Kutcha (made of mud, tin, straw)'
rdf['pucca'] = rdf.dwelling == 'Pucca (have cement/brick wall and floor'
rdf['malaria_incidence'] = rdf.malaria4months == 'Yes'
rdf['fever_incidence'] = rdf.fever4months == 'Yes'
rdf['disadvantaged_caste'] = (rdf.caste == 'SC/Dalit') | (rdf.caste == 'ST') | (rdf.caste == 'OBC')
rdf['university'] = rdf.education == 'University degree or higher'
rdf['owns_net'] = rdf.hasmosquitonet == 'Yes'

In [ ]:
d = pd.get_dummies(rdf.dwelling)
['dwelling - ' + c for c in d.columns]

In [244]:
rdf.to_csv('outs/ind-effect-for-balance.csv', index=False)

In [11]:
df = pd.read_csv('outs/ind-effect-with-assignment.csv')
userids = df[df.ind_treatment == 1].userid.astype(int).astype(str).tolist()

(4908, 43)

In [246]:
pageid = mal.confs['opt'][0].page_id

instructions = add_users_to_audience(pageid, audid, userids)

In [ ]:
len(instructions)

In [ ]:
run_instructions(instructions, mal.state)

In [72]:
import json
from facebook_business.adobjects.application import Application

def make_event(pageid, userid):
    return {
        'event': 'CUSTOM_APP_EVENTS',
        'custom_events': json.dumps([{
            '_eventName': "fb_mobile_level_achieved",
            'fb_level': 'survey_completion',
        }]),
        'advertiser_tracking_enabled': 1,
        'application_tracking_enabled': 1,
        'extinfo': json.dumps(['mb1']),
        'page_id': pageid,
        'page_scoped_user_id': userid,
    }

from toolz import curry

@curry
def send_event(app, mal, user):
    pageid = mal.confs['opt'][0].page_id
    params = make_event(pageid, user)
    return app.create_activity(params=params)

app_id = env("FACEBOOK_APP_ID")
app = Application(fbid=app_id, api=mal.state.api)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# pool = ThreadPoolExecutor(100)
# res = pool.map(send_event(app, mal), audiences[1].users[1:])

for u in audiences[1].users:
    print(u)

In [65]:
r = list(res)